In [1]:
from chemdataextractor_neel_curie import Document
from chemdataextractor_neel_curie.model import Compound
from chemdataextractor_neel_curie.doc import Paragraph, Heading, Title
from chemdataextractor.scrape import Selector
from chemdataextractor.scrape.pub.rsc import RscHtmlDocument

import os
import glob
import timeit
import pandas as pd
import collections

In [2]:
os.getcwd()

'C:\\Users\\Ayan Deep Hazra\\Desktop\\Python\\CDE_MSE_Databases\\curie_neel'

In [3]:
all_extracted_json = [] # store for database
start = timeit.default_timer()

# read all html papers
i = 0
for file in glob.glob("./test3/*.html"):
    
    #open file
    f = open(file, 'rb')
    doc = Document.from_file(f)  
    extracted_json = doc.records.serialize()#extract records # return list of dict
    for dic in extracted_json:
        print(dic)
    # Option1: if there is name and temperature extracted together
    name_temp_ls = [dc for dc in extracted_json if (('names' in dc) and ('curie_temperatures' in dc)) or (('names' in dc) and ('neel_temperatures' in dc))]

    # Option2: if temperature and name are separately extracted
    temp = [dc for dc in extracted_json if ('neel_temperatures' in dc) or ('curie_temperatures' in dc)] 
    roles = [dc for dc in extracted_json if ('roles' in dc) and (dc not in name_temp_ls)]

    #printing and storing information
    if name_temp_ls or (temp and roles): #either Option1 or Option2 is True
        

        #MetaData author, paper citations
        htmlstring = open(file,'rb').read()
        sel = Selector.from_text(htmlstring)
        scrape = RscHtmlDocument(sel)
        somedc = scrape.serialize()
        citation_dc = {'doi': somedc['doi'], 'publisher': scrape.publisher, 'title': somedc['title'], 'authors': somedc['authors'], 'filename': file} #keep only these keys
        title = somedc['title']

        print(f'{i}.\nfilename: {file}')
        print(f'Title: {title}\n')

        if name_temp_ls: # if Option 1 is True print and store Option 1
            temporary_ls = []

            for nt in name_temp_ls:
                print(nt)
                temporary_ls.append(nt)

            for r in roles: # some have exception
                print(r)
                temporary_ls.append(r)

            temporary_ls.append(citation_dc)
            all_extracted_json.append(temporary_ls)#store for database


        elif temp and roles: # if Option 2 is True print and store Option 2
            temporary_ls = []

            for r in roles: #print all names extracted with 'roles', but no temperature
                print(r)
                temporary_ls.append(r)
            for t in temp: #print all temperatures extracted, but no name proceeding it
                print(t)
                temporary_ls.append(t)

            temporary_ls.append(citation_dc)
            all_extracted_json.append(temporary_ls)#store for database

    print("----------------------------")
    i += 1
    
    if i == 25:################### testing first 50 papers
        break

stop = timeit.default_timer()
print('Run Time: ', stop - start, ' seconds')  

{'names': ['K. McBride']}
{'names': ['La1−xSrxMnO3 –']}
{'names': ['ferrites']}
{'names': ['Pd–Ni']}
{'names': ['Ni']}
{'names': ['Cu']}
{'names': ['Co']}
{'names': ['Au']}
{'names': ['Zn']}
{'names': ['manganate']}
{'names': ['perovskites']}
{'names': ['metal oxides']}
{'names': ['O2−']}
{'names': ['Fe3O4']}
{'names': ['metal oxide']}
{'names': ['metal carbonate']}
{'names': ['ammonium hydroxide']}
{'names': ['EDTA']}
{'names': ['citric acid']}
{'names': ['chelating agents']}
{'names': ['lanthanum(III) oxide']}
{'names': ['manganese(II) carbonate']}
{'names': ['strontium carbonate']}
{'names': ['H2O']}
{'names': ['ammonium hydroxide NH4OH']}
{'names': ['manganese']}
{'names': ['ammonia']}
{'names': ['Mn2+']}
{'names': ['carbon']}
{'names': ['− 0.1']}
{'names': ['O(2)']}
{'names': ['O']}
{'names': ['18e']}
{'names': ['Aztec']}
{'names': ['Sr']}
{'names': ['cobalt']}
{'names': ['MnCO3']}
{'names': ['oxygen']}
{'names': ['Sample name']}
{'names': ['H', 'SQUID']}
{'names': ['La0.8Sr0.2MnO

In [4]:
len(all_extracted_json)

1

In [5]:
all_extracted_json

[[{'names': ['Belfast BT7 1NN'],
   'curie_temperatures': [{'value': '89', 'units': '°C'}]},
  {'doi': '10.1039/c5ce01890k',
   'publisher': 'Royal Society of Chemistry',
   'title': 'Evaluation of La1−xSrxMnO3 (0 ≤ x < 0.4) synthesised via a modified sol–gel method as mediators for magnetic fluid hyperthermia',
   'authors': ['K. McBride',
    'J. Cook',
    'S. Gray',
    'S. Felton',
    'L. Stella',
    'D. Poulidi',
    'K. McBride',
    'J. Cook',
    'S. Gray',
    'S. Felton',
    'L. Stella',
    'D. Poulidi'],
   'filename': './test3\\curie_paper112.html'}]]

In [6]:
#print(""" The obtained Curie temperature value, TC, of the samples are lower than that of polycrystalline NiFe2O4 (TC = 883 K)32 and NiFe2O4 nanoparticles (TC > 800 K)69 as well as Ni0.6Zn0.4Fe2O4 nanoparticles (TC = 663 K) reported by Gawas et al. """)

In [7]:
import re

def convert_rawdata(raw_data):
    """
    Data tables created with columns including Property, Name, Raw_value, Raw_unit, Value, Unit,
    DOI, Title, Journal, Date, Specifier, Correctness, Tag, Info, and Warning.
    """
    result = list();
    
    #check if there's metadata
    meta = False
    meta_keys = ["doi", "publisher", "authors", "filename", "title"]
    for key in meta_keys:
        if key in raw_data[-1]:
            meta = True
            
    for i in range( len(raw_data)-1 ):

        keys = list(raw_data[i].keys())
        name = ", ".join(raw_data[i]["names"])

        if("curie_temperatures" in keys):
            for extracted_data in raw_data[i]["curie_temperatures"]:
                new_dict = collections.defaultdict(str)
                new_dict["Names"] = name
                new_dict["Type"] = "Curie"
                if(not "value" in extracted_data):
                    continue
                new_dict["Extracted Value"] = extracted_data["value"]
                if("units" in extracted_data):
                    new_dict["Extracted Unit"] = extracted_data["units"]
                    
                new_temp = normalise_temperature(extracted_data)
                if( new_temp != None ):
                    new_dict.update( new_temp )

                if(meta == True):
                    new_dict.update(raw_data[-1]) # metadata
                result.append(new_dict)
                new_dict['Correctness'] = 'None'
                new_dict['Tag'] = 'CDE'

        if("neel_temperatures" in keys):
            for extracted_data in raw_data[i]["neel_temperatures"]:
                new_dict = collections.defaultdict(str)
                new_dict["Names"] = name
                new_dict["Type"] = "Neel"
                if(not "value" in extracted_data):
                    continue
                new_dict["Extracted Value"] = extracted_data["value"]
                if("units" in extracted_data):
                    new_dict["Extracted Unit"] = extracted_data["units"]
                
                new_temp = normalise_temperature(extracted_data)
                if( new_temp != None ):
                    new_dict.update( new_temp )
                
                if(meta == True):
                    new_dict.update(raw_data[-1])
                result.append(new_dict)
                new_dict['Correctness'] = 'None'
                new_dict['Tag'] = 'CDE'
            
    return result

def normalise_temperature(data):
    
    try:
    
        res = dict()
        try:
            unit = str( data["units"] )
        except:
            return None

        if(data['units'] == 'K'):
            res["Normalised value"] = data["value"]
            res["Normalised units"] = 'K'
            return res

        match = re.search("c|C", data["units"])
        if(match != None): # C to K
            if('∼' in data['value']):
                front = False
                if( data['value'][0] == '∼'):
                    front = True
                divide = data["value"].split('∼')
                if(front):
                    divide = divide[1:]
                for i in range(len(divide)):
                    divide[i] = str(float(divide[i])+273)

                res["Normalised value"] = "~"+"~".join(divide)
                res["Normalised units"] = 'K'
            elif('-' in data['value'] and not data['value'][0] == '-'):
                divide = data["value"].split('-')
                for i in range(len(divide)):
                    divide[i] = str(float(divide[i])+273)
                res["Normalised value"] = "-".join(divide)
                res["Normalised units"] = 'K'
            else:
                res["Normalised value"] = float(data["value"])+273
                res["Normalised units"] = 'K'
            return res
    except:
        return None

     
    
    return None
        
        

In [8]:
database = list()
for raw_data in all_extracted_json:
    temp = convert_rawdata(raw_data)
    #print(temp)
    for clean_data in temp:
        database.append(clean_data)
        
database = pd.DataFrame(database)
database

,Names,Type,Extracted Value,Extracted Unit,Normalised value,Normalised units,doi,publisher,title,authors,filename,Correctness,Tag
0,Belfast BT7 1NN,Curie,89,°C,362.0,K,10.1039/c5ce01890k,Royal Society of Chemistry,Evaluation of La1−xSrxMnO3 (0 ≤ x < 0.4) synth...,"[K. McBride, J. Cook, S. Gray, S. Felton, L. S...",./test3\curie_paper112.html,None,CDE


In [9]:
database.to_csv("curie_neel_database_112.csv")